In [24]:
import ast
from datetime import datetime

import pandas as pd
import numpy as np

### Preprocessing Prodecure 
- Remove Duplicates and rows with NAs.
- Filter out recipes with less than n reviews (agree on n).
- Filter out users with less than n reviews (agree on n).
- map user ids and recipe ids to continuous IDS to create the user x recipe array.
- Create matrix of user-recipe ratings matrix.
- Implement 

In [6]:
# Remove Duplicates and rows with NAs
raw_interactions = pd.read_csv('RAW_interactions.csv')
print('columns', raw_interactions.columns)
print('before', raw_interactions.shape)

raw_interactions.drop(['review'], axis=1, inplace=True)
raw_interactions.dropna(inplace=True)
raw_interactions.drop_duplicates(inplace=True)
raw_interactions = raw_interactions.reindex()

print('after',raw_interactions.shape)
raw_interactions.head()

columns Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')
before (1132367, 5)
after (1132367, 4)


,user_id,recipe_id,date,rating
0,38094,40893,2003-02-17,4
1,1293707,40893,2011-12-21,5
2,8937,44394,2002-12-01,4
3,126440,85009,2010-02-27,5
4,57222,85009,2011-10-01,5


In [21]:
raw_recipes = pd.read_csv('RAW_recipes.csv')
print('columns', raw_recipes.columns)
print('before', raw_recipes.shape)

raw_recipes['tags'] = raw_recipes['tags'].apply(lambda str_tags: " ".join(ast.literal_eval(str_tags)))

raw_recipes.head()

columns Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
before (231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,60-minutes-or-less time-to-make course main-in...,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,30-minutes-or-less time-to-make course main-in...,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,time-to-make course preparation main-dish chil...,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,60-minutes-or-less time-to-make course main-in...,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,weeknight time-to-make course main-ingredient ...,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [38]:
vectorizer = TfidfVectorizer(token_pattern="(?u)\b\w\w+\b").fit(raw_recipes['tags'].values)
x = vectorizer.transform(raw_recipes['tags'])

print(vectorizer.vocabulary_)

tags_freq = {}
for tags in raw_recipes['tags'].values:
    for tag in tags.split(" "):
        if tags_freq.get(tag):
            tags_freq[tag] += 1
        else:
            tags_freq[tag] = 1



# tags_freq = sorted(tags_freq.items(), key=lambda x: x[1], reverse=True)
# print("unique tags count:", len(tags_freq))
# print("Top tags:\n", tags_freq[0:200])

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [33]:
user_pool = raw_interactions.groupby(
    "user_id", as_index=False).agg(rating_count=('rating', 'count'))
user_pool = user_pool[user_pool['rating_count'] >= 5]
print(user_pool)
# user_pool = user_pool[user_pool['rating']]


print('before:', raw_interactions.shape, 'after:', user_pool.shape)
print('removed:', raw_interactions.shape[0] - user_pool.shape[0], 'removed_precent:', ((raw_interactions.shape[0] - user_pool.shape[0]) / raw_interactions.shape[0]) * 100)

interactions = pd.merge(user_pool, raw_interactions, on='user_id', how='left')


           user_id  rating_count
0             1533           128
1             1535           794
3             1634            60
4             1676            31
7             1792            31
...            ...           ...
220883  2002157583             9
221744  2002187419             5
223000  2002227190             5
223913  2002254807             7
225155  2002295280             5

[23086 rows x 2 columns]
before: (1132367, 4) after: (23086, 2)
removed: 1109281 removed_precent: 97.96126167576413


In [31]:
print(interactions.shape)
print(raw_interactions.shape)
print('percent_removed:', (raw_interactions.shape[0] - interactions.shape[0]) / raw_interactions.shape[0])
interactions.head()

(872021, 5)
(1132367, 4)
percent_removed: 0.22991309354652686


,user_id,rating_count,recipe_id,date,rating
0,1533,128,116345,2005-02-16,5
1,1533,128,32907,2005-05-21,5
2,1533,128,14750,2005-05-18,5
3,1533,128,24136,2002-05-20,5
4,1533,128,63598,2003-09-23,4


In [34]:
user_ids = interactions['user_id'].unique()
user_indexes = np.arange(len(user_ids))

user2idx = dict(zip(user_ids, user_indexes))
idx2user = dict(zip(user_indexes, user_ids))

In [35]:
recipe_ids = interactions['recipe_id'].unique()
recipe_indexes = np.arange(len(recipe_ids))

recipe2idx = dict(zip(recipe_ids, recipe_indexes))
idx2recipe = dict(zip(recipe_indexes, recipe_ids))

In [36]:
interactions['user_idx'] = interactions['user_id'].apply(lambda x: user2idx[x])
interactions['recipe_idx'] = interactions['recipe_id'].apply(lambda x: recipe2idx[x])
interactions['rating'] = interactions['rating'].apply(lambda x: float(x > 0))

In [37]:
interactions['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
interactions.head()

,user_id,rating_count,recipe_id,date,rating,user_idx,recipe_idx
0,1533,128,116345,2005-02-16,1.0,0,0
1,1533,128,32907,2005-05-21,1.0,0,1
2,1533,128,14750,2005-05-18,1.0,0,2
3,1533,128,24136,2002-05-20,1.0,0,3
4,1533,128,63598,2003-09-23,1.0,0,4


In [107]:
df.head(10)

,user_id,recipe_id,date,rating,user_idx,recipe_idx
1099,2046,4684,2000-02-25,1.0,7,1094
1101,2046,517,2000-02-25,1.0,7,1096
1103,2046,4523,2000-02-25,1.0,7,1098
1104,2046,3431,2000-04-07,1.0,7,1099
1105,2046,13307,2000-05-21,1.0,7,1100
...,...,...,...,...,...,...
853855,2002093000,56804,2018-12-16,1.0,19799,18967
519981,334014,3655,2018-12-17,1.0,8896,162741
663345,527509,78552,2018-12-17,1.0,12167,12685
849489,1800292505,509363,2018-12-18,1.0,19541,208820
